In [1]:
%cd ../../spell_correction

/Users/tyoyo/lab/spell_correction


In [2]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-large")
tokenizer = T5Tokenizer.from_pretrained("google/mt5-large")
for p in model.parameters():
    p.requires_grad = False

## Multilingualなsubword分割がどれくらい日本語で使えそうか
* "平安"や"申し上げ"などが分割されているあたり、ある程度しっかり日本語対応できている（以前のMulti-lingual BERTではほぼcharベースだった）
* 例外として"すもももももももものうち"は"もも"で分割されてしまうので意味的にはあまり良くない分割になっている

In [3]:
'", "'.join(tokenizer.tokenize('すもももももももものうち、とは平安時代に徳川家康が申し上げたものであるといわれている。'))

'▁", "す", "もも", "もも", "もも", "もも", "のうち", "、", "とは", "平安", "時代に", "徳", "川", "家", "康", "が", "申し上げ", "た", "ものである", "といわれ", "ている", "。'

In [4]:
'", "'.join(tokenizer.tokenize("本記事では Google の T5(Text-to-Text Transfer Transformer) *1によるテキスト生成について、学習や推論のコード例と実験結果を交えてご紹介します。実験としては livedoor ニュースコーパス*2での文章分類、やさしい日本語コーパス*3及びやさしい日本語拡張コーパス*4を用いたやさしい日本語変換を行いました。"))

'▁", "本", "記事", "では", "▁Google", "▁", "の", "▁T", "5", "(", "Text", "-", "to", "-", "Text", "▁Transfer", "▁Transformer", ")", "▁", "*1", "による", "テキスト", "生成", "について", "、", "学習", "や", "推", "論", "の", "コード", "例", "と", "実験", "結果を", "交", "えて", "ご紹介します", "。", "実験", "としては", "▁live", "door", "▁", "ニュース", "コー", "パス", "*", "2", "での", "文章", "分類", "、", "やさしい", "日本語", "コー", "パス", "*", "3", "及び", "やさしい", "日本語", "拡張", "コー", "パス", "*", "4", "を用いた", "やさしい", "日本語", "変換", "を行いました", "。'

## mT5-largeの言語モデリングの性能をチェック

In [5]:
text = "安倍晋三は日本の <extra_id_0> だ。"
outputs = model.generate(tokenizer(text, return_tensors='pt').input_ids)
tokenizer.convert_ids_to_tokens(outputs.numpy().tolist()[0])

['<pad>',
 '▁<extra_id_0>',
 '▁',
 '総理',
 '▁<extra_id_1>',
 '▁',
 '総理',
 '▁<extra_id_2>',
 '▁',
 '総理',
 '▁<extra_id_3>',
 '▁',
 '総理',
 '▁<extra_id_4>',
 '▁',
 '総理',
 '▁<extra_id_5>',
 '▁',
 '総理',
 '▁<extra_id_6>']

In [6]:
text = "安倍晋三は日本の <extra_id_0> で、 <extra_id_1> と呼ばれる経済政策を打ち出し日本経済を大きく回復させた。"
outputs = model.generate(tokenizer(text, return_tensors='pt').input_ids)
tokenizer.convert_ids_to_tokens(outputs.numpy().tolist()[0])

['<pad>',
 '▁<extra_id_0>',
 '▁',
 '首相',
 '▁<extra_id_1>',
 '▁「',
 'アベ',
 'ノミ',
 'クス',
 '」',
 '▁<extra_id_2>',
 '▁',
 '経済',
 '政策',
 'の',
 '▁',
 'リーダー',
 '▁<extra_id_3>',
 '▁',
 '首相']

In [7]:
# ❌な例, wiki
text = " <extra_id_0>はグラフ理論における最短経路問題を解くための最良優先探索によるアルゴリズムである。"
outputs = model.generate(tokenizer(text, return_tensors='pt').input_ids)
tokenizer.convert_ids_to_tokens(outputs.numpy().tolist()[0])

['<pad>',
 '▁<extra_id_0>',
 'である',
 '。',
 '▁',
 '▁<extra_id_1>',
 'ゴ',
 'リズム',
 '▁<extra_id_1>',
 'ゴ',
 'リズム',
 '▁<extra_id_15>',
 '知',
 'られている',
 '。',
 '▁',
 '最も',
 '重要な',
 '特徴',
 'は']

In [8]:
# ⭕️な例, yahoo news
text = "売れっ子 <extra_id_0>だけに、豪邸で優雅な暮らしをしているのでは？　と思いきや、 <extra_id_1>を送っている。"
outputs = model.generate(tokenizer(text, return_tensors='pt').input_ids)
tokenizer.convert_ids_to_tokens(outputs.numpy().tolist()[0])

['<pad>',
 '▁<extra_id_0>',
 '女優',
 '▁<extra_id_1>',
 '実際には',
 '、',
 '平凡',
 'な',
 '生活',
 '▁<extra_id_2>',
 '芸能人',
 'は',
 '、',
 '平凡',
 'な',
 '生活',
 '▁<extra_id_3>',
 '女優',
 'な',
 '▁<extra_id_15>']

In [9]:
# ⭕️, 5ch
# "ｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗ ゴミゲーを１０円で買ったからって１０円の損失やってのｗオマエｗ分かってねえなｗｗｗｗｗ"
text = "ｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗｗ ゴミゲーを１０円で買った <extra_id_0>１０円の損失やってのｗ <extra_id_1>ｗ分かってねえな <extra_id_2>"
outputs = model.generate(tokenizer(text, return_tensors='pt').input_ids)
tokenizer.convert_ids_to_tokens(outputs.numpy().tolist()[0])

['<pad>',
 '▁<extra_id_0>',
 'のに',
 '▁<extra_id_1>',
 '▁',
 '何',
 'やって',
 'んの',
 '▁<extra_id_2>',
 'w',
 '</s>']

In [10]:
# ⭕️, twitter
# "しやがれ中ずっと 泣くもんかって涙堪えてたんだけど、HELLO NEW DREAMのcm流れてダメだった。素敵なサプライズ....もう愛しかなくて....すごいよ嵐。スポンサーの皆さんありがとう号泣"
text = "しやがれ中ずっと 泣くもんかって涙堪えてたんだけど、HELLO NEW DREAMのcm流れて <extra_id_0>だった。素敵な <extra_id_1>もう愛しかなくて....すごいよ嵐。スポンサーの皆さん <extra_id_2>号泣"
outputs = model.generate(tokenizer(text, return_tensors='pt').input_ids)
tokenizer.convert_ids_to_tokens(outputs.numpy().tolist()[0])

['<pad>',
 '▁<extra_id_0>',
 '、',
 'もう',
 '涙',
 'が止ま',
 'らない',
 'くらい',
 '▁<extra_id_1>',
 '歌',
 '声',
 'で',
 '▁<extra_id_2>',
 'ありがとう',
 '。',
 '▁<extra_id_3>',
 '、',
 'もう',
 '涙',
 'が止ま']

# mT5でzero-shotなスペル修正を行う
検証すること
1. Recallの高いエラー検出ができたとして、「decodingの各ステップで編集距離による制約を満たす最も尤度の高いトークンを選択していく」ことでエラー修正ができそうかどうかを確認
2. mT5のdecoderを使ってスペルエラー検出ができそうかどうか

## 1. エラー検出ができたとして、エラー修正ができそうかどうか

In [11]:
texts = [
    "彼はファヴローにクラッシック西部劇映画を提供した。", # 'クラッシック' → 'クラシック'
    "民主党の予備選挙ではヒラリー・クリントンが1位となり、バラク・オバマが僅差の2位だた。",  # 'だた' → 'だった'
    "クラゲには4つ放射管を持つが触手はもたない。",  # '' -> 'の'
    "桶狭間の戦いで国友制の鉄砲が織田信長により初めて戦力として使用された。",  # '制' → '製'
    "ひきこもりの更正をはじめたきっかけ",  # '正' → '生'
]    

In [12]:
# 何かしらのエラー検出でマスクする
masked_texts = [
    "彼はファヴローに <extra_id_0>西部劇映画を提供した。", # 'クラッシック' → 'クラシック'
    "民主党の予備選挙ではヒラリー・クリントンが1位となり、バラク・オバマが僅差の2位 <extra_id_0>",  # 'だた' → 'だった'
    "クラゲには <extra_id_0>を持つが触手はもたない。",  # '' -> 'の'
    "桶狭間の戦いで <extra_id_0>の鉄砲が織田信長により初めて戦力として使用された。",  # '制' → '製'
    "ひきこもりの <extra_id_0>をはじめたきっかけ",  # '正' → '生'
]

In [13]:
batch = tokenizer.prepare_seq2seq_batch(src_texts=masked_texts, tgt_texts=['<pad> <extra_id_0>'] * 5, return_tensors="pt")
output_step1 = model(**batch)

In [14]:
output_step1 = model(**batch)

In [15]:
output_step1.logits.shape

torch.Size([5, 3, 250112])

In [17]:
top_logits, top_indices = output_step1.logits[:, -1].sort(dim=-1, descending=True)  # 末尾のトークンの次にくる単語の予測をスコアでソート

In [18]:
print(top_logits.shape, top_indices.shape)

torch.Size([5, 250112]) torch.Size([5, 250112])


In [19]:
top_tokens = [tokenizer.convert_ids_to_tokens(each_indices) for each_indices in top_indices.numpy().tolist()]

In [23]:
from spell_correction.utils import BeginningLevenshtein
beginning_levenshtein = BeginningLevenshtein()

In [59]:
import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

### 'クラッシック'に代わる尤もらしい単語

原文: 彼はファヴローにクラッシック西部劇映画を提供した。

* 単純に尤度が高い単語を選ぶと、文意を損なう。
* 尤度が高く、かつ'ク'から始まるトークンを選ぶとスペル修正に使えそう
  * 厳密には'クラッシック'の冒頭部との編集距離が小さく、よりマッチしているトークン（詳細は後述）

In [74]:
original_token = 'クラッシック'
candidates = []
for rank, (token, logit, index) in enumerate(zip(top_tokens[0], top_logits[0], top_indices[0])):
    candidate = beginning_levenshtein(token, original_token)
    candidate['rank'] = rank
    candidate['token'] = token
    candidate['logit'] = logit.tolist()
    candidate['index'] = index.tolist()
    # より一致部分が多く、より似ていて、より尤度の高いトークンを選びたい多目的最適化のようなものだが、暫定的に適当なスコアを作って最大化で代用する
    candidate['match_score'] = candidate['match_len'] / (candidate['distance'] + 1) * sigmoid(candidate['logit'])
    candidates.append(candidate)

In [78]:
for c in candidates[:30]:
    print(c)

{'match_ref': '', 'match_len': 0, 'distance': 5.0, 'rank': 0, 'token': 'いくつかの', 'logit': 3.3284339904785156, 'index': 176894, 'match_score': 0.0}
{'match_ref': '', 'match_len': 0, 'distance': 2.0, 'rank': 1, 'token': '彼の', 'logit': 2.6667046546936035, 'index': 104567, 'match_score': 0.0}
{'match_ref': '', 'match_len': 0, 'distance': 3.0, 'rank': 2, 'token': '多くの', 'logit': 2.545552968978882, 'index': 36431, 'match_score': 0.0}
{'match_ref': '', 'match_len': 0, 'distance': 1.0, 'rank': 3, 'token': '、', 'logit': 1.5796977281570435, 'index': 292, 'match_score': 0.0}
{'match_ref': '', 'match_len': 0, 'distance': 3.0, 'rank': 4, 'token': '新しい', 'logit': 0.9527663588523865, 'index': 26634, 'match_score': 0.0}
{'match_ref': '', 'match_len': 0, 'distance': 3.0, 'rank': 5, 'token': '最初の', 'logit': 0.588798999786377, 'index': 100843, 'match_score': 0.0}
{'match_ref': '', 'match_len': 0, 'distance': 2.0, 'rank': 6, 'token': '最も', 'logit': 0.4598374366760254, 'index': 65454, 'match_score': 0.0}
{'

In [79]:
for c in sorted(candidates, key=lambda c: c['match_score'], reverse=True)[:30]:
    print(c)

{'match_ref': 'クラッシック', 'match_len': 6, 'distance': 1.0, 'rank': 569, 'token': 'クラシック', 'logit': -3.036130905151367, 'index': 119003, 'match_score': 0.1374600984655472}
{'match_ref': 'クラ', 'match_len': 2, 'distance': 3.0, 'rank': 103, 'token': 'フランス', 'logit': -1.4590768814086914, 'index': 75179, 'match_score': 0.09430427720012638}
{'match_ref': 'クラッ', 'match_len': 3, 'distance': 4.0, 'rank': 324, 'token': 'ヨーロッパ', 'logit': -2.505263566970825, 'index': 122950, 'match_score': 0.045294004325828884}
{'match_ref': 'ク', 'match_len': 1, 'distance': 0.0, 'rank': 758, 'token': 'ク', 'logit': -3.3221492767333984, 'index': 6662, 'match_score': 0.0348191060957602}
{'match_ref': 'クラッ', 'match_len': 3, 'distance': 2.0, 'rank': 885, 'token': 'ブラック', 'logit': -3.479574203491211, 'index': 21545, 'match_score': 0.02989902759181323}
{'match_ref': 'クラッシ', 'match_len': 4, 'distance': 6.0, 'rank': 500, 'token': 'アニメーション', 'logit': -2.90417218208313, 'index': 235395, 'match_score': 0.02968440124392427}
{'mat

In [81]:
# 別のスコア関数を試してみる (distanceにexpをかけることで、編集距離を重視)
# 2番目の候補が'ク'になっていていい感じ
original_token = 'クラッシック'
candidates = []
for rank, (token, logit, index) in enumerate(zip(top_tokens[0], top_logits[0], top_indices[0])):
    candidate = beginning_levenshtein(token, original_token)
    candidate['rank'] = rank
    candidate['token'] = token
    candidate['logit'] = logit.tolist()
    candidate['index'] = index.tolist()
    # より一致部分が多く、より似ていて、より尤度の高いトークンを選びたい多目的最適化のようなものだが、暫定的に適当なスコアを作って最大化で代用する
    candidate['match_score'] = candidate['match_len'] / math.exp(candidate['distance']) * sigmoid(candidate['logit'])
    candidates.append(candidate)
for c in sorted(candidates, key=lambda c: c['match_score'], reverse=True)[:30]:
    print(c)

{'match_ref': 'クラッシック', 'match_len': 6, 'distance': 1.0, 'rank': 569, 'token': 'クラシック', 'logit': -3.036130905151367, 'index': 119003, 'match_score': 0.10113748841375388}
{'match_ref': 'ク', 'match_len': 1, 'distance': 0.0, 'rank': 758, 'token': 'ク', 'logit': -3.3221492767333984, 'index': 6662, 'match_score': 0.0348191060957602}
{'match_ref': 'クラ', 'match_len': 2, 'distance': 3.0, 'rank': 103, 'token': 'フランス', 'logit': -1.4590768814086914, 'index': 75179, 'match_score': 0.01878053398537874}
{'match_ref': 'ク', 'match_len': 1, 'distance': 1.0, 'rank': 703, 'token': 'クリ', 'logit': -3.253140926361084, 'index': 68250, 'match_score': 0.013690334116179878}
{'match_ref': 'クラッ', 'match_len': 3, 'distance': 2.0, 'rank': 885, 'token': 'ブラック', 'logit': -3.479574203491211, 'index': 21545, 'match_score': 0.012139180102912022}
{'match_ref': 'クラ', 'match_len': 2, 'distance': 2.0, 'rank': 634, 'token': 'ドラマ', 'logit': -3.1350512504577637, 'index': 53130, 'match_score': 0.011282728824212239}
{'match_ref':

In [83]:
# "民主党の予備選挙ではヒラリー・クリントンが1位となり、バラク・オバマが僅差の2位 <extra_id_0>",  # 'だた' → 'だった'
original_token = 'だた'
candidates = []
for rank, (token, logit, index) in enumerate(zip(top_tokens[0], top_logits[0], top_indices[0])):
    candidate = beginning_levenshtein(token, original_token)
    candidate['rank'] = rank
    candidate['token'] = token
    candidate['logit'] = logit.tolist()
    candidate['index'] = index.tolist()
    # より一致部分が多く、より似ていて、より尤度の高いトークンを選びたい多目的最適化のようなものだが、暫定的に適当なスコアを作って最大化で代用する
    candidate['match_score'] = candidate['match_len'] / (candidate['distance'] + 1) * sigmoid(candidate['logit'])
    candidates.append(candidate)
for c in sorted(candidates, key=lambda c: c['match_score'], reverse=True)[:30]:
    print(c)

{'match_ref': 'だた', 'match_len': 2, 'distance': 2.0, 'rank': 7, 'token': '優れた', 'logit': 0.37967419624328613, 'index': 163464, 'match_score': 0.39586301375129107}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 72, 'token': 'また', 'logit': -1.1446471214294434, 'index': 6134, 'match_score': 0.2414681659188452}
{'match_ref': 'だた', 'match_len': 2, 'distance': 3.0, 'rank': 42, 'token': 'あなたの', 'logit': -0.6287240982055664, 'index': 42283, 'match_score': 0.17389995046254675}
{'match_ref': 'だた', 'match_len': 2, 'distance': 3.0, 'rank': 43, 'token': '書かれた', 'logit': -0.6733708381652832, 'index': 234721, 'match_score': 0.1688712325681762}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 140, 'token': '似た', 'logit': -1.7438538074493408, 'index': 224733, 'match_score': 0.14882409064086052}
{'match_ref': 'だた', 'match_len': 2, 'distance': 2.0, 'rank': 80, 'token': '新たな', 'logit': -1.2902559041976929, 'index': 87989, 'match_score': 0.14387299968778164}
{'match_ref': 'だた', 'm

In [82]:
# 別のスコア関数を試してみる (distanceにexpをかけることで、編集距離を重視)
# "民主党の予備選挙ではヒラリー・クリントンが1位となり、バラク・オバマが僅差の2位 <extra_id_0>",  # 'だた' → 'だった'
original_token = 'だた'
candidates = []
for rank, (token, logit, index) in enumerate(zip(top_tokens[0], top_logits[0], top_indices[0])):
    candidate = beginning_levenshtein(token, original_token)
    candidate['rank'] = rank
    candidate['token'] = token
    candidate['logit'] = logit.tolist()
    candidate['index'] = index.tolist()
    # より一致部分が多く、より似ていて、より尤度の高いトークンを選びたい多目的最適化のようなものだが、暫定的に適当なスコアを作って最大化で代用する
    candidate['match_score'] = candidate['match_len'] / math.exp(candidate['distance']) * sigmoid(candidate['logit'])
    candidates.append(candidate)
for c in sorted(candidates, key=lambda c: c['match_score'], reverse=True)[:30]:
    print(c)

{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 72, 'token': 'また', 'logit': -1.1446471214294434, 'index': 6134, 'match_score': 0.17766234787783577}
{'match_ref': 'だた', 'match_len': 2, 'distance': 2.0, 'rank': 7, 'token': '優れた', 'logit': 0.37967419624328613, 'index': 163464, 'match_score': 0.16072269926679025}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 140, 'token': '似た', 'logit': -1.7438538074493408, 'index': 224733, 'match_score': 0.1094986465956157}
{'match_ref': 'だた', 'match_len': 2, 'distance': 2.0, 'rank': 80, 'token': '新たな', 'logit': -1.2902559041976929, 'index': 87989, 'match_score': 0.05841327948854106}
{'match_ref': 'だた', 'match_len': 2, 'distance': 2.0, 'rank': 100, 'token': '書いた', 'logit': -1.4306073188781738, 'index': 142012, 'match_score': 0.052240522191365914}
{'match_ref': 'だ', 'match_len': 1, 'distance': 1.0, 'rank': 221, 'token': 'まだ', 'logit': -2.154733419418335, 'index': 27363, 'match_score': 0.03821889605573615}
{'match_ref': 'だた', 'ma

In [86]:
# 別のスコア関数を試してみる (distanceにexpをかけることで、編集距離を重視)
# "民主党の予備選挙ではヒラリー・クリントンが1位となり、バラク・オバマが僅差の2位 <extra_id_0>",  # 'だた' → 'だった'
original_token = 'だた'
candidates = []
for rank, (token, logit, index) in enumerate(zip(top_tokens[0], top_logits[0], top_indices[0])):
    candidate = beginning_levenshtein(token, original_token)
    candidate['rank'] = rank
    candidate['token'] = token
    candidate['logit'] = logit.tolist()
    candidate['index'] = index.tolist()
    # より一致部分が多く、より似ていて、より尤度の高いトークンを選びたい多目的最適化のようなものだが、暫定的に適当なスコアを作って最大化で代用する
    candidate['match_score'] = candidate['match_len'] / math.exp(candidate['distance']) * sigmoid(candidate['logit'])
    candidates.append(candidate)

In [89]:
for c in sorted(filter(lambda c: c['distance'] <= 1, candidates), key=lambda c: c['match_score'], reverse=True)[:100]:
    print(c)

{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 72, 'token': 'また', 'logit': -1.1446471214294434, 'index': 6134, 'match_score': 0.17766234787783577}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 140, 'token': '似た', 'logit': -1.7438538074493408, 'index': 224733, 'match_score': 0.1094986465956157}
{'match_ref': 'だ', 'match_len': 1, 'distance': 1.0, 'rank': 221, 'token': 'まだ', 'logit': -2.154733419418335, 'index': 27363, 'match_score': 0.03821889605573615}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 688, 'token': 'いた', 'logit': -3.2325279712677, 'index': 10963, 'match_score': 0.027929275107408506}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 960, 'token': 'した', 'logit': -3.568343162536621, 'index': 2166, 'match_score': 0.020181112133137315}
{'match_ref': 'だた', 'match_len': 2, 'distance': 1.0, 'rank': 997, 'token': '見た', 'logit': -3.6060240268707275, 'index': 113704, 'match_score': 0.019454552255527224}
{'match_ref': 'だ', 'match_l

In [90]:
# "クラゲには <extra_id_0>を持つが触手はもたない。"
# '' -> 'の'
original_token = '4つ放射管'
candidates = []
for rank, (token, logit, index) in enumerate(zip(top_tokens[2], top_logits[2], top_indices[2])):
    candidate = beginning_levenshtein(token, original_token)
    candidate['rank'] = rank
    candidate['token'] = token
    candidate['logit'] = logit.tolist()
    candidate['index'] = index.tolist()
    # より一致部分が多く、より似ていて、より尤度の高いトークンを選びたい多目的最適化のようなものだが、暫定的に適当なスコアを作って最大化で代用する
    candidate['match_score'] = candidate['match_len'] / (candidate['distance'] + 1) * sigmoid(candidate['logit'])
    candidates.append(candidate)
for c in sorted(candidates, key=lambda c: c['match_score'], reverse=True)[:30]:
    print(c)

{'match_ref': '4', 'match_len': 1, 'distance': 0.0, 'rank': 263, 'token': '4', 'logit': -2.3533935546875, 'index': 410, 'match_score': 0.08679641236419319}
{'match_ref': '4つ', 'match_len': 2, 'distance': 2.0, 'rank': 195, 'token': '二つの', 'logit': -2.083512544631958, 'index': 228290, 'match_score': 0.07380644856269011}
{'match_ref': '4つ', 'match_len': 2, 'distance': 1.0, 'rank': 387, 'token': 'まつ', 'logit': -2.7072627544403076, 'index': 165235, 'match_score': 0.06254615535134471}
{'match_ref': '4つ', 'match_len': 2, 'distance': 3.0, 'rank': 271, 'token': 'いくつか', 'logit': -2.3756659030914307, 'index': 153442, 'match_score': 0.042523606990237}
{'match_ref': '4つ', 'match_len': 2, 'distance': 1.0, 'rank': 630, 'token': '一つ', 'logit': -3.158996343612671, 'index': 87573, 'match_score': 0.04073825703582776}
{'match_ref': '4つ', 'match_len': 2, 'distance': 4.0, 'rank': 259, 'token': 'いくつかの', 'logit': -2.3410849571228027, 'index': 176894, 'match_score': 0.035110800425335466}
{'match_ref': '4つ', 'm

In [92]:
#  "桶狭間の戦いで国友制の鉄砲が織田信長により初めて戦力として使用された。"
# 桶狭間の戦いで <extra_id_0>の鉄砲が織田信長により初めて戦力として使用された。"
# '制' → '製'
original_token = '国友制'
candidates = []
for rank, (token, logit, index) in enumerate(zip(top_tokens[3], top_logits[3], top_indices[3])):
    candidate = beginning_levenshtein(token, original_token)
    candidate['rank'] = rank
    candidate['token'] = token
    candidate['logit'] = logit.tolist()
    candidate['index'] = index.tolist()
    # より一致部分が多く、より似ていて、より尤度の高いトークンを選びたい多目的最適化のようなものだが、暫定的に適当なスコアを作って最大化で代用する
    candidate['match_score'] = candidate['match_len'] / (candidate['distance'] + 1) * sigmoid(candidate['logit'])
    candidates.append(candidate)
for c in sorted(candidates, key=lambda c: c['match_score'], reverse=True)[:30]:
    print(c)

{'match_ref': '国', 'match_len': 1, 'distance': 1.0, 'rank': 103, 'token': '中国', 'logit': -3.304124355316162, 'index': 3161, 'match_score': 0.017714985167892324}
{'match_ref': '国', 'match_len': 1, 'distance': 1.0, 'rank': 296, 'token': '我国', 'logit': -4.302565574645996, 'index': 44635, 'match_score': 0.006676537367965645}
{'match_ref': '国', 'match_len': 1, 'distance': 1.0, 'rank': 322, 'token': '英国', 'logit': -4.3750386238098145, 'index': 41573, 'match_score': 0.006215588336775192}
{'match_ref': '国', 'match_len': 1, 'distance': 0.0, 'rank': 617, 'token': '国', 'logit': -5.123970985412598, 'index': 3802, 'match_score': 0.005917118553079561}
{'match_ref': '国', 'match_len': 1, 'distance': 1.0, 'rank': 622, 'token': '四国', 'logit': -5.126492023468018, 'index': 159421, 'match_score': 0.002951153997625095}
{'match_ref': '国', 'match_len': 1, 'distance': 2.0, 'rank': 431, 'token': '中国の', 'logit': -4.740740776062012, 'index': 176582, 'match_score': 0.0028855281303586445}
{'match_ref': '国', 'match_

In [93]:
# "ひきこもりの <extra_id_0>をはじめたきっかけ",  # '正' → '生'
original_token = '更正'
candidates = []
for rank, (token, logit, index) in enumerate(zip(top_tokens[4], top_logits[4], top_indices[4])):
    candidate = beginning_levenshtein(token, original_token)
    candidate['rank'] = rank
    candidate['token'] = token
    candidate['logit'] = logit.tolist()
    candidate['index'] = index.tolist()
    # より一致部分が多く、より似ていて、より尤度の高いトークンを選びたい多目的最適化のようなものだが、暫定的に適当なスコアを作って最大化で代用する
    candidate['match_score'] = candidate['match_len'] / (candidate['distance'] + 1) * sigmoid(candidate['logit'])
    candidates.append(candidate)
for c in sorted(candidates, key=lambda c: c['match_score'], reverse=True)[:30]:
    print(c)

{'match_ref': '更', 'match_len': 1, 'distance': 0.0, 'rank': 2436, 'token': '更', 'logit': -4.3974151611328125, 'index': 9411, 'match_score': 0.012159443894496878}
{'match_ref': '更', 'match_len': 1, 'distance': 1.0, 'rank': 1639, 'token': '更新', 'logit': -3.7998902797698975, 'index': 16449, 'match_score': 0.010941809672227546}
{'match_ref': '更正', 'match_len': 2, 'distance': 1.0, 'rank': 2827, 'token': '矯正', 'logit': -4.664065361022949, 'index': 190284, 'match_score': 0.00933999791265149}
{'match_ref': '更正', 'match_len': 2, 'distance': 1.0, 'rank': 5894, 'token': '修正', 'logit': -6.107279300689697, 'index': 88101, 'match_score': 0.0022216537544695702}
{'match_ref': '更正', 'match_len': 2, 'distance': 1.0, 'rank': 7644, 'token': '大正', 'logit': -6.761699676513672, 'index': 213565, 'match_score': 0.001155922828935303}
{'match_ref': '更正', 'match_len': 2, 'distance': 1.0, 'rank': 8000, 'token': '適正', 'logit': -6.863402843475342, 'index': 176439, 'match_score': 0.0010442590916993914}
{'match_ref': 

# Idea Memo

検出ができなくても、テキストを10分割くらいにして、各セグメントをマスクして直すということを10回やればいいや！！ ！
以内稲井
つまり、「編集距離を一定以下に保ったまま、refinement（マスクして、尤度を高くする）をし続ける」

スペル修正というタスクを、

* 目的関数：尤度の最大化
* 拘束条件: 編集距離の差が一定値以内 など

というタスクに置き換える！

拘束条件は言語や状況によって変えてもいい
* 読み/拼音の近さ
* 文字の視覚的な類似度
  * 実装としては「Visual Edit Distance」的な感じ。編集距離の各編集に重み付けをする
    * l -> I のコストは0.1
    * l -> A のコストは1.0
    * l -> φ (削除)のコストは1.0
    * . -> φ (削除)のコストは0.5

拘束条件を、編集距離の重みで表現したらスマートな実装になりそう

## やってみた感想
* ただ単純にマスクして予測させて、編集距離で拘束条件づけるだけだと、モデルが修正前の単語を知らないから候補が絞りきれない
  * 更正 を 更生 に直したいが、更新などが候補として上がってきてしまう。
    * これとかなら尤度で弾きうるが、国友制〜 → 国友製〜 が　中国制の~ とかになってしまいうる。
* マスクすると修正前の単語の情報が伝わらないのでなんか別の方法を考える

In [5]:
text = "民主党の予備選挙ではヒラリー・クリントンが1位となり、バラク・オバマが僅差の2位 <extra_id_0> (だた)"
outputs = model.generate(tokenizer(text, return_tensors='pt').input_ids)
tokenizer.convert_ids_to_tokens(outputs.numpy().tolist()[0])

['<pad>',
 '▁<extra_id_0>',
 'となりました',
 '。',
 '▁<extra_id_1>',
 'となりました',
 '。',
 '▁',
 '-',
 '▁',
 'だ',
 'た',
 '▁<extra_id_15>',
 'となりました',
 '。',
 '▁',
 '-',
 '▁',
 'だ',
 'た']

In [6]:
batch = tokenizer.prepare_seq2seq_batch(src_texts=["民主党の予備選挙ではヒラリー・クリントンが1位となり、バラク・オバマが僅差の2位 <extra_id_0>"], tgt_texts=['<pad> <extra_id_0>だ'], return_tensors="pt")

In [7]:
output_step1 = model(**batch)

In [8]:
top_logits, top_indices = output_step1.logits[:, -1].sort(dim=-1, descending=True)  # 末尾のトークンの次にくる単語の予測をスコアでソート
top_tokens = [tokenizer.convert_ids_to_tokens(each_indices) for each_indices in top_indices.numpy().tolist()]

In [9]:
top_tokens[:10]

[['。',
  '。(',
  'と言われています',
  '...',
  'っています',
  '。。',
  '。【',
  'そうだ',
  '」',
  '。■',
  '。”',
  '▁',
  '。[',
  '、',
  '。※',
  'そう',
  '!',
  '.',
  'と思っています',
  '。」',
  '▁→',
  '。・',
  '▁■',
  'と言われ',
  '......',
  '(',
  'となりました',
  '。「',
  'といいます',
  '・・・',
  'しています',
  'た',
  'そうです',
  '៕',
  'しました',
  '▁...',
  '▁<extra_id_1>',
  'といわれ',
  'っている',
  '...。',
  '。●',
  '】',
  '”',
  '・・・。',
  'ということです',
  'だ',
  '</s>',
  'と言えます',
  'と感じました',
  '....',
  'となっています',
  'として',
  'そうで',
  'という',
  'っ',
  '・・',
  '〕',
  'と言う',
  'した',
  '■',
  '】【。】',
  '¡£',
  '・・。',
  '▁<extra_id_56>',
  '[/',
  'といえる',
  'ました',
  'と言って',
  '▼',
  'と思われます',
  '"',
  '。1',
  'と言った',
  'と言われている',
  '▁<extra_id_18>',
  '</',
  'とした',
  'となります',
  'となった',
  'る',
  '▁>>',
  '\u202c',
  'でした',
  'と思いました',
  '!!',
  '」(',
  'のです',
  'と思われ',
  'まし',
  'と思っている',
  'と言われて',
  '【',
  '」。',
  '▁[...]',
  ')',
  'していました',
  '.....',
  '▁៕',
  'ま',
  '▁↑',
  '[',
  'と考えています',
  '”。',
  'ます',
  'します',
  ')。',
  '――